<a href="https://colab.research.google.com/github/Maulik2604/Project----Face-Mask-Detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project -> Face Mask Detection Using Yolo V7

In [4]:
!pip install -q kaggle

In [7]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection

In [ ]:
!unzip "/content/face-mask-detection.zip" -d "/content/face-mask-detection/"

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

In [ ]:
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss0.png")

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss1.png")

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss100.png")

In [ ]:
im1 = Image.open(r"/content/face-mask-detection/images/maksssksksss100.png")
im2 = ImageOps.grayscale(im1)

im2

In [16]:
def xml_to_yolo_bbox(bbox, w, h):

    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h

    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h

    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):

    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2

    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)

    return [xmin, ymin, xmax, ymax]

classes = []

input_dir = "/content/face-mask-detection/annotations"
output_dir = "/content/labels"
image_dir = "/content/face-mask-detection/images"

os.mkdir(output_dir)

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

import glob

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue

    result = []


    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        label = obj.find("name").text


        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)


        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:

        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))

with open("/content/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In [ ]:
with open('/content/classes.txt') as f:
    contents = f.read()
    print(contents)

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss0.png")

In [ ]:
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

In [ ]:
with open('/content/labels/maksssksksss0.txt') as f:
    contents = f.read()
    print(contents)

In [21]:
os.mkdir("/content/data/")
os.mkdir('/content/data/train')
os.mkdir('/content/data/val')
os.mkdir('/content/data/test')
os.mkdir('/content/data/train/images')
os.mkdir('/content/data/train/labels')
os.mkdir('/content/data/test/images')
os.mkdir('/content/data/test/labels')
os.mkdir('/content/data/val/images')
os.mkdir('/content/data/val/labels')

In [22]:
metarial = []

for i in os.listdir("/content/face-mask-detection/images"):
    srt = i[:-4]
    metarial.append(srt)

In [ ]:
len(metarial)

In [ ]:
metarial[0:10]

In [25]:
def preparinbdata(main_txt_file, main_img_file, train_size, test_size, val_size):

    for i in range(0,train_size):

        source_txt = main_txt_file + "/" + metarial[i] + ".txt"
        source_img = main_img_file + "/" + metarial[i] + ".png"

        mstring = metarial[i]
        train_destination_txt = "/content/data/train/labels" + "/" + metarial[i] + ".txt"
        train_destination_png = "/content/data/train/images" + "/" + metarial[i] + ".png"

        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)


    for l in range(train_size , train_size + test_size):

        source_txt = main_txt_file + "/" + metarial[l] + ".txt"
        source_img = main_img_file + "/" + metarial[l] + ".png"

        mstring = metarial[l]
        test_destination_txt = "/content/data/test/labels" + "/" + metarial[l] + ".txt"
        test_destination_png = "/content/data/test/images" + "/" + metarial[l] + ".png"

        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_img, test_destination_png)



    for n in range(train_size + test_size , train_size + test_size + val_size):

        source_txt = main_txt_file + "/" + metarial[n] + ".txt"
        source_img = main_img_file + "/" + metarial[n] + ".png"

        mstring = metarial[n]
        val_destination_txt = "/content/data/val/labels" + "/" + metarial[n] + ".txt"
        val_destination_png = "/content/data/val/images" + "/" + metarial[n] + ".png"

        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png)



In [26]:
preparinbdata("/content/labels", "/content/face-mask-detection/images", 603, 150, 100)

In [ ]:
%cd /content/data

yaml_text = """train: /content/data/train/images
val: /content/data/val/images

nc: 3
names: ["without_mask", "with_mask", "mask_weared_incorrect"]"""

with open("/content/data/data.yaml", 'w') as file:
    file.write(yaml_text)

%cat data/data.yaml

In [ ]:
%cd /content/
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
%cd /content/

In [ ]:
# Train
!python /content/yolov7/train.py --workers 8 --device 0 --batch-size 16 --epochs 50 --data /content/data/data.yaml  --cfg /content/yolov7/cfg/training/yolov7.yaml --weights '' --name yolov7_1 --hyp /content/yolov7/data/hyp.scratch.p5.yaml

In [ ]:
Image.open("/content/runs/train/yolov7_1/results.png")

In [ ]:
Image.open("/content/runs/train/yolov7_1/train_batch0.jpg")

In [ ]:
Image.open("/content/runs/train/yolov7_1/train_batch9.jpg")

In [ ]:
!# Detection
!python /content/yolov7/detect.py --weights /content/runs/train/yolov7_1/weights/best.pt --conf 0.25 --img-size 640 --source /content/data/test/images/

In [ ]:
Image.open("/content/runs/detect/exp/maksssksksss1.png")

In [ ]:
Image.open("/content/runs/detect/exp/maksssksksss508.png")

In [ ]:
# I downloaded one image for use at the detect.
%cd /content
!wget "https://onecms-res.cloudinary.com/image/upload/s--XV7DHKzY--/c_fill,g_auto,h_468,w_830/f_auto,q_auto/people-wearing-mask-at-orchard-road-singapore-feb-3--49-.jpg?itok=GdDk1T6A"

In [ ]:
!# Detection
!python //content//yolov7//detect.py --weights /content/model/model_r101_2_class.pt --conf 0.25 --img-size 640 --source //content//12_58_5000.png

In [ ]:
Image.open("/content/runs/detect/exp3/image0.jpg")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/model_r101_2_class.zip" -d "/content/model/"